In [1]:
"""
[V7]
Blend 7 Models:
* kibuna-nn-hs-1024-last-train (aka. 2stage-NN, LB: 0.01822)
* simpleNN-oldcv (LB: 0.01836)
* deepinsight-efficientnet-v7-b3-infer (LB: 0.01850)
* deepinsight_resnest_lightning_v2_infer (LB: 0.01854)
* deepinsight_resnest_lightning_v1_infer (LB: 0.01853)
* fork-of-2heads-looper-super-puper-markpeng-infer (LB: 0.1836)
* improving-mark-s-2-heads-model-infer.py (LB: 0.1844)

Removed for now due to low weights:
* 503-203-tabnet-with-nonscored-features (LB: 0.01836)


"""

kernel_mode = False

import os
import numpy as np
import pandas as pd
import time
import random
import math
import datetime
import pickle
from pickle import dump, load
import glob

from numba import njit
from scipy.optimize import minimize, fsolve

import optuna

import warnings
warnings.filterwarnings('ignore')

import gc
gc.enable()

rand_seed = 1120

search_mode = True
run_submit_script = False

# method = "CV"
# method = "scipy_per_target"
# method = "scipy"
method = "optuna"
study_name = "blend_search_optuna_v7_10folds"
# study_name = "blend_search_optuna_v7"
# study_name = "blend_search_optuna_v6_per_target"
# n_trials = 500
# n_trials = 3000
# n_trials = 5000
n_trials = 10000

In [2]:
# !mkdir -p /root/.cache/torch/hub/checkpoints/
# !cp ../input/gen-efficientnet-pretrained/tf_efficientnet_*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v1-resnest50/*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v2-resnest50-output/*.pth /root/.cache/torch/hub/checkpoints/
# !ls -la /root/.cache/torch/hub/checkpoints/

In [3]:
# !cp ../input/kaggle-moa-team/scripts/* .
# !cp ../input/kaggle-moa-team/blends/*.pkl .
# !ls -la

In [4]:
dataset_folder = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA/"

# Add your model inference script here
# Tuple Format: (script, oof_filename, output_filename, weight)
model_list = [
    # 10 folds
    ("2stageNN_with_ns_oldcv_10folds.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv_10folds.py",
     "../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv_10folds.npy",
     "submission_2stageNN_with_ns_oldcv_10folds.csv",
     0),
    
    ("script_simpleNN_oldcv_10folds.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv_10folds.py",
     "../../Github/kaggle_moa_team/oof/oof_NN_oldcv_10fold.npy",
     "submission_script_simpleNN_oldcv_10folds.csv",
     0),
    
    #     ("2stageNN_with_ns_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py",
    #      "../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy",
    #      "submission_2stageNN_with_ns_oldcv_0.01822.csv",
    #      0.323528084383917),

    #     ("script_simpleNN_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py",
    #      "../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy",
    #      "submission_script_simpleNN_oldcv_0.01836.csv",
    #      0.08786476491118465),
    
    #     ("fork-of-2heads-looper-super-puper-markpeng-infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/fork-of-2heads-looper-super-puper-markpeng-infer.py",
    #      "../../Github/kaggle_moa_team/oof/oof_fork-of-2heads-looper-super-puper-markpeng.npy",
    #      "submission_2heads-looper-super-puper_0.01836.csv",
    #      0.018966959973949222),
    
    ("improving-mark-s-2-heads-model-infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py",
     "../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy",
     "submission_improving-mark-s-2-heads-model.csv",
     0),
    
    ("deepinsight_efficientnet_lightning_v7_b3_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy",
     "submission_effnet_v7_b3_0.01850.csv",
     0.21849845883367852),
    
    ("deepinsight_resnest_lightning_v1_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v1_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy",
     "submission_resnest_v1_0.01853.csv",
     0.3704230222796271),
    
    ("deepinsight_resnest_lightning_v2_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy",
     "submission_resnest_v2_0.01854.csv",
     0.3704230222796271),
    
    #     ("script_tabnet_ns_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_tabnet_ns_oldcv.py",
    #      "../../Github/kaggle_moa_team/oof/oof_tabnet_ns_oldcv.npy",
    #      "submission_tabnet_ns_oldcv_0.01836.csv",
    #      0.0013224625996093413),
]

model_path = "." if kernel_mode else dataset_folder

In [5]:
train_features = pd.read_csv(f"{dataset_folder}/train_features.csv",
                             engine='c')
train_labels = pd.read_csv(f'{dataset_folder}/train_targets_scored.csv',
                           engine='c')
train_classes = [c for c in train_labels.columns if c != "sig_id"]

non_control_group_rows = train_features["cp_type"] == "trt_cp"
non_control_group_train_labels = train_labels.loc[
    non_control_group_rows, :].copy().reset_index(drop=True)

submission = pd.read_csv(f'{dataset_folder}/sample_submission.csv')
submission.iloc[:, 1:] = 0

In [6]:
def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.nanmean(-logloss)


def save_pickle(obj, folder, name):
    dump(obj, open(f"{folder}/{name}.pkl", 'wb'), pickle.HIGHEST_PROTOCOL)


def load_pickle(path):
    return load(open(path, 'rb'))

In [7]:
# Reference: https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
# CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
def log_loss_numpy(y_pred, y_true):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, -np.log(y_pred),
                    -np.log(1 - y_pred))
    return loss.mean()

def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, all_oof, axes=((0), (0)))
    return log_loss_numpy(oof_blend, y_true)

@njit
def grad_func_jit(weights):
    oof_clip = np.minimum(1 - 1e-15, np.maximum(all_oof, 1e-15))
    gradients = np.zeros(all_oof.shape[0])
    for i in range(all_oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros(
            (all_oof.shape[1], all_oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean(
            (-a * b + (b**2) * weights[i] + b * c) /
            ((b**2) *
             (weights[i]**2) + 2 * b * c * weights[i] - b * weights[i] +
             (c**2) - c))
    return gradients

## TPE (Tree-structured Parzen Estimator) and Sequential Least Squares Programming (SLSQP)
https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler

https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html

In [8]:
def run_inference_scripts(submission, weights=None, target_weights=None):
    for i, (script, oof_filename, output_filename, weight) in enumerate(model_list):
        print(f"Generating submission file from {script} ......")
        infer_start = time.time()
        !python {model_path}/{script}
        infer_elapsed = time.time() - infer_start
        print(f"Time spent on inference: {infer_elapsed/60:.2f} minutes.")

        model_submit = pd.read_csv(output_filename, engine='c')
        print(model_submit.head(5))
        print(model_submit.shape)

        if target_weights is not None:
            for j, target in enumerate(train_classes):
                print(f"Blending {script} for {target} with weight: {optimized_target_weights[j][i]} ......")
                submission.iloc[:, j+1] += model_submit.iloc[:, j+1] * optimized_target_weights[j][i]
        elif weights is None:
            print(f"Blending {script} with weight: {weight} ......")
            submission.iloc[:, 1:] += weight * model_submit.iloc[:, 1:]
        else:
            print(f"Blending {script} with weight: {weights[i]} ......")
            submission.iloc[:, 1:] += weights[i] * model_submit.iloc[:, 1:]

    return submission

In [ ]:
total_start = time.time()

if not search_mode and run_submit_script:
    if method == "scipy_per_target":
        weights_path = glob.glob(f'{model_path}/{study_name}_*.pkl')[0]
        print(f"Loading target-wise optimized weights from {weights_path} ......")
        optimized_target_weights = load_pickle(weights_path)

        # For 206 target weights
        submission = run_inference_scripts(
            submission, target_weights=optimized_target_weights)
    else:
        submission = run_inference_scripts(submission)

elif search_mode and method == "CV":
    y_true = non_control_group_train_labels[train_classes].values

    all_oof = np.zeros(
        (len(model_list), non_control_group_train_labels.shape[0], 206))
    blend_oof = np.zeros((non_control_group_train_labels.shape[0], 206))
    print(all_oof.shape)
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        all_oof[i, :, :] = oof
        blend_oof += oof * weight

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")

    blend_oof_loss = mean_logloss(blend_oof, y_true)
    print(f"Blend OOF Validation Loss: {blend_oof_loss:.6f}\n")

elif search_mode and method == "optuna":
    print("[Optuna]")
    ## Search Best Blend Weights by Optuna ##
    model_oofs = []

    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        oof_loss = mean_logloss(
            oof, non_control_group_train_labels[train_classes].values)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    def objective(trial):
        weights = []
        for i in range(len(model_list)):
            weights.append(trial.suggest_float(f"w{i}", 0, 1.0))

        blend = np.zeros(model_oofs[0].shape)
        for i in range(len(model_list)):
            blend += weights[i] * model_oofs[i]
        blend = np.clip(blend, 0, 1.0)

        loss = mean_logloss(
            blend, non_control_group_train_labels[train_classes].values)
        return loss

    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=0,
        interval_steps=1,
    )
    sampler = optuna.samplers.TPESampler(seed=rand_seed)
    study = optuna.create_study(direction="minimize",
                                pruner=pruner,
                                sampler=sampler,
                                study_name=study_name,
                                storage=f'sqlite:///{study_name}.db',
                                load_if_exists=True)

    study.optimize(objective,
                   n_trials=n_trials,
                   timeout=None,
                   gc_after_trial=True,
                   n_jobs=-1)

    trial = study.best_trial

    if run_submit_script:
        optimal_weights = []
        for i, (script, oof_filename, output_filename,
                _) in enumerate(model_list):
            optimal_weights.append(trial.params[f"w{i}"])
        submission = run_inference_scripts(submission, weights=optimal_weights)

    print("\n[Optuna]")
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

elif search_mode and method == "scipy":
    print("[Scipy SLSQP]")
    # Optimise Blending Weights with Bonus
    # https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
    model_oofs = []
    y_true = non_control_group_train_labels[train_classes].values

    all_oof = np.zeros(
        (len(model_list), non_control_group_train_labels.shape[0], 206))
    print(all_oof.shape)
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        all_oof[i, :, :] = oof

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    tol = 1e-10
    init_guess = [1 / all_oof.shape[0]] * all_oof.shape[0]
    bnds = [(0, 1) for _ in range(all_oof.shape[0])]
    cons = {
        'type': 'eq',
        'fun': lambda x: np.sum(x) - 1,
        'jac': lambda x: [1] * len(x)
    }

    print('Inital Blend OOF:', func_numpy_metric(init_guess))

    start_time = time.time()
    res_scipy = minimize(
        fun=func_numpy_metric,
        x0=init_guess,
        method='SLSQP',
        # jac=grad_func_jit,  # grad_func
        bounds=bnds,
        constraints=cons,
        tol=tol)
    print("\n[Scipy SLSQP]")
    print(
        f'[{str(datetime.timedelta(seconds = time.time() - start_time))[2:7]}] Optimised Blend OOF:',
        res_scipy.fun)
    print(f'Optimised Weights: {res_scipy.x}\n')

    if run_submit_script:
        submission = run_inference_scripts(submission, weights=res_scipy.x)

# Target-wise Weight Optimization #

elif search_mode and method == "scipy_per_target":
    print("[Scipy SLSQP]")
    # Optimise Blending Weights with Bonus
    # https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
    model_oofs = []
    y_true = non_control_group_train_labels[train_classes].values

    all_oof = np.zeros(
        (len(model_list), non_control_group_train_labels.shape[0], 206))
    print(all_oof.shape)
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        all_oof[i, :, :] = oof

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    print("\n[Scipy SLSQP Per Target]")
    optimized_target_weights = []
    for i, target in enumerate(train_classes):
        tol = 1e-10
        init_guess = [1 / all_oof.shape[0]] * all_oof.shape[0]
        bnds = [(0, 1) for _ in range(all_oof.shape[0])]
        cons = {
            'type': 'eq',
            'fun': lambda x: np.sum(x) - 1,
            'jac': lambda x: [1] * len(x)
        }

        def func_numpy_metric_targes(weights):
            oof_blend = np.tensordot(weights,
                                     all_oof[:, :, i],
                                     axes=((0), (0)))
            return log_loss_numpy(oof_blend, y_true[:, i])

        start_time = time.time()
        res_scipy = minimize(
            fun=func_numpy_metric_targes,
            x0=init_guess,
            method='SLSQP',
            # jac=grad_func_jit,  # grad_func
            bounds=bnds,
            constraints=cons,
            tol=tol)

        print(
            f'[{str(datetime.timedelta(seconds = time.time() - start_time))[2:7]}] ' + \
            f'Optimised Blend OOF for {target}:', res_scipy.fun)
        print(f'Optimised Weights for {target}: {res_scipy.x}\n')
        optimized_target_weights.append(res_scipy.x)

    blend_targets_oof = np.zeros(
        (non_control_group_train_labels.shape[0], 206))
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        for j in range(206):
            blend_targets_oof[:,
                              j] += oof[:, j] * optimized_target_weights[j][i]

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")

    blend_targets_oof_loss = mean_logloss(blend_targets_oof, y_true)
    print(
        f"Blend Target-Wise OOF Validation Loss: {blend_targets_oof_loss:.6f}\n"
    )

    # Save optimized weights per target
    save_pickle(optimized_target_weights, model_path,
                f"{study_name}_{blend_targets_oof_loss}")

    if run_submit_script:
        # For 206 target weights
        submission = run_inference_scripts(
            submission, target_weights=optimized_target_weights)

[Optuna]
Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv_10folds.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv_10folds.py: 0.015461

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_NN_oldcv_10fold.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py: 0.015741

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
OOF Validation Loss of ../../Github/

[I 2020-11-30 16:25:29,624] A new study created in RDB with name: blend_search_optuna_v7_10folds


OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756



[I 2020-11-30 16:25:30,255] Trial 1 finished with value: 0.018815197403747676 and parameters: {'w0': 0.1268625027806528, 'w1': 0.10558424091800733, 'w2': 0.039714620423227154, 'w3': 0.3981370009666264, 'w4': 0.4485180512658108, 'w5': 0.7451906385255455}. Best is trial 1 with value: 0.018815197403747676.
[I 2020-11-30 16:25:30,548] Trial 3 finished with value: 0.020401799140911305 and parameters: {'w0': 0.163864597290858, 'w1': 0.5419403291149064, 'w2': 0.07062670435430862, 'w3': 0.8009653809476306, 'w4': 0.6417495209027022, 'w5': 0.0037652784036432685}. Best is trial 1 with value: 0.018216147422329663.
[I 2020-11-30 16:25:30,551] Trial 2 finished with value: 0.018216147422329663 and parameters: {'w0': 0.45280646108985945, 'w1': 0.14436756576371057, 'w2': 0.5016726345624218, 'w3': 0.4216547856235645, 'w4': 0.1857104095132731, 'w5': 0.14196585499674919}. Best is trial 2 with value: 0.018216147422329663.
[I 2020-11-30 16:25:30,581] Trial 0 finished with value: 0.026296439468524786 and par

[I 2020-11-30 16:25:33,834] Trial 27 finished with value: 0.016529088612640744 and parameters: {'w0': 0.06272142122236891, 'w1': 0.22732440113112334, 'w2': 0.32922731647226366, 'w3': 0.2959417869550685, 'w4': 0.3513039931937042, 'w5': 0.14976295706809298}. Best is trial 13 with value: 0.015268680523445604.
[I 2020-11-30 16:25:33,869] Trial 26 finished with value: 0.016345209257605554 and parameters: {'w0': 0.06408460632255643, 'w1': 0.4861006626837874, 'w2': 0.2924806756790496, 'w3': 0.003085530126536923, 'w4': 0.36094640911945186, 'w5': 0.15562319645243106}. Best is trial 13 with value: 0.015268680523445604.
[I 2020-11-30 16:25:34,087] Trial 28 finished with value: 0.01727712605952048 and parameters: {'w0': 0.0654088494232368, 'w1': 0.4927380879116813, 'w2': 0.295282751619569, 'w3': 0.3073835387703951, 'w4': 0.3019305316975267, 'w5': 0.1495291834451834}. Best is trial 13 with value: 0.015268680523445604.
[I 2020-11-30 16:25:34,104] Trial 30 finished with value: 0.017373763158479157 an

...

[I 2020-11-30 16:29:58,967] Trial 1508 finished with value: 0.015118244179292122 and parameters: {'w0': 0.26736257452744827, 'w1': 0.0183720985845435, 'w2': 0.10083068872351583, 'w3': 0.16473286981555807, 'w4': 0.1516097427693282, 'w5': 0.27680309075867787}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:29:59,110] Trial 1509 finished with value: 0.015227034411459435 and parameters: {'w0': 0.3091765937911423, 'w1': 0.06452768149342827, 'w2': 0.09645906285803485, 'w3': 0.1614303455501136, 'w4': 0.15886968366175802, 'w5': 0.2761394349904914}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:29:59,356] Trial 1511 finished with value: 0.015130282493217741 and parameters: {'w0': 0.3073916585302253, 'w1': 0.018440325089988892, 'w2': 0.10081631957315841, 'w3': 0.16655384702428158, 'w4': 0.15504275175783133, 'w5': 0.27373091300925734}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:29:59,407] Trial 1510 finished with value: 0.0

[I 2020-11-30 16:30:04,028] Trial 1536 finished with value: 0.015158508153933287 and parameters: {'w0': 0.31257230796743635, 'w1': 0.018724185017912388, 'w2': 0.08643115196321449, 'w3': 0.15629905981649486, 'w4': 0.09211040906944742, 'w5': 0.23004229676260005}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:04,037] Trial 1533 finished with value: 0.015159467057791264 and parameters: {'w0': 0.3142764859035842, 'w1': 0.018376377394462542, 'w2': 0.08847851493357417, 'w3': 0.1528792415899931, 'w4': 0.08778847319572977, 'w5': 0.23329779700531975}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:04,054] Trial 1538 finished with value: 0.01516879956271011 and parameters: {'w0': 0.3123775365954814, 'w1': 0.017059240285286957, 'w2': 0.08415572771078295, 'w3': 0.1486013348417246, 'w4': 0.08623697461305663, 'w5': 0.2347768075226753}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:04,060] Trial 1534 finished with value: 0

[I 2020-11-30 16:30:08,811] Trial 1559 finished with value: 0.015124417919251858 and parameters: {'w0': 0.3133824303153532, 'w1': 0.07108435285066653, 'w2': 0.10582358722879563, 'w3': 0.1407738789698126, 'w4': 0.16647324890398285, 'w5': 0.21225931101692785}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:08,820] Trial 1562 finished with value: 0.015121841923648372 and parameters: {'w0': 0.3566185430437311, 'w1': 0.05858441540215126, 'w2': 0.03893084267808586, 'w3': 0.14774163930359033, 'w4': 0.16369073499616302, 'w5': 0.20558837881775266}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:08,824] Trial 1561 finished with value: 0.015133575302268455 and parameters: {'w0': 0.31648063803690446, 'w1': 0.07036032605032691, 'w2': 0.10006176864250681, 'w3': 0.1423290284838702, 'w4': 0.10078658033103124, 'w5': 0.2153413125970645}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:08,834] Trial 1563 finished with value: 0.0

[I 2020-11-30 16:30:13,467] Trial 1587 finished with value: 0.015128858402827386 and parameters: {'w0': 0.35151620921941695, 'w1': 0.031574759838949584, 'w2': 0.06905745972719146, 'w3': 0.1274896750690132, 'w4': 0.13426920011424406, 'w5': 0.23265912580899364}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:13,470] Trial 1586 finished with value: 0.015260865281512107 and parameters: {'w0': 0.35450891182460526, 'w1': 0.03249209280777764, 'w2': 0.0630723009904339, 'w3': 0.13121758752903956, 'w4': 0.13718359494593393, 'w5': 0.0906200897396861}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:13,476] Trial 1588 finished with value: 0.015249015189491703 and parameters: {'w0': 0.3538928747079753, 'w1': 0.0336263513513619, 'w2': 0.03232261230505439, 'w3': 0.1340421895565627, 'w4': 0.19779707716346961, 'w5': 0.31728640218029336}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:13,484] Trial 1589 finished with value: 0.0

[I 2020-11-30 16:30:18,727] Trial 1611 finished with value: 0.015345111687421994 and parameters: {'w0': 0.3533771169405307, 'w1': 0.0002724513007373592, 'w2': 0.11659840723651202, 'w3': 0.14813905886650414, 'w4': 0.18287895644745988, 'w5': 0.29630623630181874}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:18,730] Trial 1613 finished with value: 0.015726347076443728 and parameters: {'w0': 0.3718733193981145, 'w1': 0.038608568083492265, 'w2': 0.11966932718490048, 'w3': 0.20307856202283675, 'w4': 0.1818419965600187, 'w5': 0.2950732838566785}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:18,747] Trial 1612 finished with value: 0.015582285077899853 and parameters: {'w0': 0.3546274436374701, 'w1': 0.07486918105816551, 'w2': 0.11828370639795802, 'w3': 0.14570132737316058, 'w4': 0.17700781316148284, 'w5': 0.298297294205811}. Best is trial 1361 with value: 0.015108327958135001.
[I 2020-11-30 16:30:18,763] Trial 1615 finished with value: 0.

In [ ]:
total_elapsed = time.time() - total_start
print(f"Total time spent: {total_elapsed/60:.2f} minutes.")

In [ ]:
# [V7 - without TabNet, Mark's 2heads, 10-folds 2StageNN and SimpleNN]


# [Scipy SLSQP]
# [00:48] Optimised Blend OOF: 0.015107004381942337
# Optimised Weights: [0.36727479 0.         0.08130163 0.17091322 0.10863643 0.27187394]

In [ ]:
# [V7 - without TabNet, Mark's 2heads]
# [Optuna]
# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py: 0.015606

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py: 0.015846

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v1_infer.py: 0.015819

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756

# [Optuna]
# Number of finished trials: 10000
# Best trial:
#   Value: 0.015146055466812214
#   Params: 
#     w0: 0.2590485239176548
#     w1: 0.033637086725830254
#     w2: 0.12194872432858535
#     w3: 0.19075874431151257
#     w4: 0.11084230101870868
#     w5: 0.28295491196215444

# [Scipy SLSQP]
# [00:30] Optimised Blend OOF: 0.01514578929579469
# Optimised Weights: [0.25256016 0.03296526 0.13166163 0.18166774 0.11849445 0.28265076]

In [ ]:
# [V7 - without TabNet]
# [Optuna]
# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py: 0.015606

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py: 0.015846

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_fork-of-2heads-looper-super-puper-markpeng.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/fork-of-2heads-looper-super-puper-markpeng-infer.py: 0.015887

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v1_infer.py: 0.015819

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756

# Number of finished trials: 10000
# Best trial:
#   Value: 0.015146827058312776
#   Params: 
#     w0: 0.2507759947969869
#     w1: 0.03339376719044784
#     w2: 8.528382280836113e-05
#     w3: 0.11625864567289637
#     w4: 0.18770116032490702
#     w5: 0.09530145379465743
#     w6: 0.3164264877487286

# [Scipy SLSQP]
# [00:45] Optimised Blend OOF: 0.015145788791365362
# Optimised Weights: [2.52672054e-01 3.32734786e-02 1.05430191e-17 1.30957268e-01
#  1.82060938e-01 1.18387855e-01 2.82648406e-01]

In [ ]:
# [V6 - without TabNet, 2heads]
# Total time spent: 0.68 minutes.
# Blend Target-Wise OOF Validation Loss: 0.015044

In [ ]:
# [V6 - without TabNet, 2heads]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015171999561900233
#   Params: 
#     w0: 0.323528084383917
#     w1: 0.08786476491118465
#     w2: 0.21849845883367852
#     w3: 0.3704230222796271

# [Scipy SLSQP]
# [00:22] Optimised Blend OOF: 0.015172004593585666
# Optimised Weights: [0.32020133 0.09043987 0.22122948 0.36812932]

In [ ]:
# [V6 - without TabNet]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015172424601530761
#   Params: 
#     w0: 0.3138176484100186
#     w1: 0.07850519440561339
#     w2: 0.0007183363099561991
#     w3: 0.23849563017967007
#     w4: 0.3694870328388392

# [Scipy SLSQP]
# [00:21] Optimised Blend OOF: 0.015172004898867827
# Optimised Weights: [0.32045559 0.09026525 0.         0.22069638 0.36858278]

In [ ]:
# [V6]
# [Optuna]
# Number of finished trials: 5000
# Best trial:
#   Value: 0.015173437622007157
#   Params: 
#     w0: 0.30923325055652684
#     w1: 0.09831493504786226
#     w2: 0.018966959973949222
#     w3: 0.19863369862866234
#     w4: 0.0013224625996093413
#     w5: 0.3728865483320761

# [Scipy SLSQP]
# [00:36] Optimised Blend OOF: 0.015172005464591968
# Optimised Weights: [3.20472642e-01 9.01191588e-02 1.78893358e-18 2.20448482e-01
#  3.27971157e-18 3.68959717e-01]

In [ ]:
# [V5]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015344701181290615
#   Params: 
#     w0: 0.5141433844379889
#     w1: 0.11747776562133813
#     w2: 0.3668324643717302

# [00:14] Optimised Blend OOF: 0.015344695215068541
# Optimised Weights: [0.51922623 0.11292509 0.36784869]

In [ ]:
# [V4]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015331901615194453
#   Params: 
#     w0: 0.4505928450756189
#     w1: 0.13010257032841785
#     w2: 0.06308933354044946
#     w3: 0.35639153615958885
#
# [Scipy]
# [00:23] Optimised Blend OOF: 0.015331777381591449
# Optimised Weights: [0.44090106 0.14508641 0.05945655 0.35455598]

In [ ]:
# [V3]
# improving-mark-s-2-heads-model-infer
# Number of finished trials: 3000
# Best trial:
#   Value: 0.01515466145873492
#   Params: 
#     w0: 0.0002980690037490555
#     w1: 0.29771381784976886
#     w2: 0.1569191862042946
#     w3: 0.18156875605872544
#     w4: 0.36371774630338105

In [ ]:
# [V3]
# fork-of-2heads-looper-super-puper-markpeng-infer
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015170138066049686
#   Params: 
#     w0: 0.00019903389488299251
#     w1: 0.3853752127955825
#     w2: 0.015968332256452233
#     w3: 0.22945916769823432
#     w4: 0.3711290150522236

In [ ]:
if search_mode and method == "scipy_per_target":
    # OOF scores per target
    target_oof_losses = []
    for i, target in enumerate(train_classes):
        print(target)
        # print(y_true[:, i])

        oof_loss = mean_logloss(blend_targets_oof[:, i], y_true[:, i])
        target_oof_losses.append(oof_loss)
        print(f"Blend OOF Validation Loss of {target}: {oof_loss:.6f}\n")

    target_loss_df = pd.DataFrame(
        data={
            "target": train_classes,
            "oof_logloss": target_oof_losses
        },
        columns=["target", "oof_logloss"
                 ]).sort_values(by="oof_logloss",
                                ascending=False).reset_index(drop=True)
    print(target_loss_df)

In [ ]:
if run_submit_script:
    print(submission.shape)
    print(submission)
    submission.to_csv('submission.csv', index=False)

## EOF

In [ ]:
if kernel_mode:
    !rm ./*.py
    !ls -la